<a href="https://colab.research.google.com/github/viliturpeinen/Colabs/blob/main/TEHTAVA7_KOE27112025_PAATOKSENTEKOMALLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- Peliteoriaan perustuvat päätöskriteerit yrityksen A strategian valintaan ---
# Tässä koodissa on *valmiiksi* sisällä antamasi matriisi + todennäköisyydet.

import numpy as np
import pandas as pd

# --------------------------------------
# 1. PAYOFF-MATRIISI (Yritys A, S1–S3)
# --------------------------------------
# Rivit = S1, S2, S3
# Sarakkeet = C1, C2, C3, C4

payoff_matrix = np.array([
    [ 6, -4, 10, -11],   # S1
    [ 5,  4, -11, 5],    # S2
    [-2,  0,  -8, 8],    # S3
], dtype=float)

# --------------------------------------
# 2. HISTORIAPOHJAISET TODENNÄKÖISYYDET
# --------------------------------------
probs = np.array([0.25, 0.40, 0.15, 0.20])  # C1, C2, C3, C4

# --------------------------------------
# 3. KRITEERIFUNKTIOT
# --------------------------------------

def wald_maximin(matrix):
    row_mins = matrix.min(axis=1)
    best = np.argmax(row_mins)
    return row_mins, best

def hurwicz(matrix, alpha):
    row_max = matrix.max(axis=1)
    row_min = matrix.min(axis=1)
    values = alpha * row_max + (1 - alpha) * row_min
    best = np.argmax(values)
    return values, best

def laplace(matrix, probs=None):
    if probs is None:
        probs = np.ones(matrix.shape[1]) / matrix.shape[1]
    expected = matrix.dot(probs)
    best = np.argmax(expected)
    return expected, best

def savage(matrix):
    col_max = matrix.max(axis=0)
    regret = col_max - matrix
    max_regret = regret.max(axis=1)
    best = np.argmin(max_regret)
    return regret, max_regret, best

def expected_values(matrix, probs):
    expv = matrix.dot(probs)
    best = np.argmax(expv)
    return expv, best

# --------------------------------------
# 4. LASKENTA
# --------------------------------------

strategy_names = ["S1", "S2", "S3"]
state_names = ["C1", "C2", "C3", "C4"]

df = pd.DataFrame(payoff_matrix, index=strategy_names, columns=state_names)
print("=== Payoff-matriisi ===")
display(df)

# Wald
wald_vals, wald_choice = wald_maximin(payoff_matrix)

# Hurwicz (alpha 0.6 tai muuta tarvittaessa)
alpha = 0.6
hur_vals, hur_choice = hurwicz(payoff_matrix, alpha)

# Laplace (oletus: käytetään annettuja todennäköisyyksiä)
lap_vals, lap_choice = laplace(payoff_matrix, probs)

# Savage (regret)
regret_matrix, regrets, sav_choice = savage(payoff_matrix)

# Historiapohjaiset odotusarvot
exp_vals, exp_choice = expected_values(payoff_matrix, probs)

# --------------------------------------
# 5. TULOSTUS
# --------------------------------------

print("\n=== Wald (Maximin) ===")
for i,v in enumerate(wald_vals):
    print(f"{strategy_names[i]}: min = {v}")
print("Suositus:", strategy_names[wald_choice])

print("\n=== Hurwicz (alpha =", alpha, ") ===")
for i,v in enumerate(hur_vals):
    print(f"{strategy_names[i]}: Hurwicz-arvo = {v}")
print("Suositus:", strategy_names[hur_choice])

print("\n=== Laplace/Odotusarvo tasajakaumalla tai annetuilla todennäköisyyksillä ===")
for i,v in enumerate(lap_vals):
    print(f"{strategy_names[i]}: odotusarvo = {v}")
print("Suositus:", strategy_names[lap_choice])

print("\n=== Savage (Minimax Regret) ===")
display(pd.DataFrame(regret_matrix, index=strategy_names, columns=state_names))
for i,v in enumerate(regrets):
    print(f"{strategy_names[i]}: maksimi-regret = {v}")
print("Suositus:", strategy_names[sav_choice])

print("\n=== Historiapohjaiset odotusarvot (25%, 40%, 15%, 20%) ===")
for i,v in enumerate(exp_vals):
    print(f"{strategy_names[i]}: EV = {v}")
print("Suositus:", strategy_names[exp_choice])

# Lopuksi yhteenveto
print("\n=== YHTEENVETO SUOSITUKSISTA ===")
print("Wald:", strategy_names[wald_choice])
print("Hurwicz:", strategy_names[hur_choice])
print("Laplace:", strategy_names[lap_choice])
print("Savage:", strategy_names[sav_choice])
print("Historiapohjainen EV:", strategy_names[exp_choice])


=== Payoff-matriisi ===


,C1,C2,C3,C4
S1,6.0,-4.0,10.0,-11.0
S2,5.0,4.0,-11.0,5.0
S3,-2.0,0.0,-8.0,8.0



=== Wald (Maximin) ===
S1: min = -11.0
S2: min = -11.0
S3: min = -8.0
Suositus: S3

=== Hurwicz (alpha = 0.6 ) ===
S1: Hurwicz-arvo = 1.5999999999999996
S2: Hurwicz-arvo = -1.4000000000000004
S3: Hurwicz-arvo = 1.5999999999999996
Suositus: S1

=== Laplace/Odotusarvo tasajakaumalla tai annetuilla todennäköisyyksillä ===
S1: odotusarvo = -0.8000000000000003
S2: odotusarvo = 2.2
S3: odotusarvo = -0.09999999999999987
Suositus: S2

=== Savage (Minimax Regret) ===


,C1,C2,C3,C4
S1,0.0,8.0,0.0,19.0
S2,1.0,0.0,21.0,3.0
S3,8.0,4.0,18.0,0.0


S1: maksimi-regret = 19.0
S2: maksimi-regret = 21.0
S3: maksimi-regret = 18.0
Suositus: S3

=== Historiapohjaiset odotusarvot (25%, 40%, 15%, 20%) ===
S1: EV = -0.8000000000000003
S2: EV = 2.2
S3: EV = -0.09999999999999987
Suositus: S2

=== YHTEENVETO SUOSITUKSISTA ===
Wald: S3
Hurwicz: S1
Laplace: S2
Savage: S3
Historiapohjainen EV: S2
